In [1]:
from langchain_core.prompts import ChatPromptTemplate

from enums import CollectionNames, EmbeddingModels
from langchain_ollama import ChatOllama
from langchain_core.documents import Document

from retrieval_util import (
    get_qdrant_client,
    get_embeddings_function,
    retrieve_with_score_without_filters
)

import os
import re
import logging
from typing import List, Tuple

os.environ[
    "TOKENIZERS_PARALLELISM"] = "false"  # Verhindert Fehler beim Erzeugen von Embeddings (Tritt sporadisch auf, konnte nicht gezielt reproduziert werden)
logging.basicConfig(level=logging.INFO)

Der Code ist nahezu identisch zu dem aus dem antworten_generieren.ipynb, jedoch werden hier 5 andere Fragen genutzt, deren Antworten nicht im Kontext enthalten sind.


In [2]:
# Konstanten
EMBEDDINGS = get_embeddings_function(model_name=EmbeddingModels.MIXEDBREAD.value)
QDRANT_URL = "http://localhost:6333"
COLLECTION = CollectionNames.MIXEDBREAD_RECURSIVE.value

FRAGEN = {
    1: "Wer ist der Präsident der USA?",
    2: "Wie funktioniert der HNSW Algorithmus?",
    3: "Welche Algorithmen eignen sich für die Suche in Vektordatenbanken?",
    4: "Ist die FH morgen geschlossen?",
    5: "Welche Studiengänge werden an der Ruhr-Universität Bochum angeboten?",
}
LLAMA_MODELL = "llama3.1:8b"

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-16 12:09:08,173 [INFO] PyTorch version 2.6.0 available.
2025-04-16 12:09:08,515 [INFO] Use pytorch device_name: mps
2025-04-16 12:09:08,515 [INFO] Load pretrained SentenceTransformer: mixedbread-ai/deepset-mxbai-embed-de-large-v1
2025-04-16 12:09:11,812 [INFO] 2 prompts are loaded, with the keys: ['query', 'passage']


In [3]:
FH_PROMPT = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Du bist der Assistent der Fachhochschule Südwestfalen (FH-SWF).
Du beantwortest Fragen von Studieninteressierten und Studierenden.

Nutze den bereitgestellten Kontext, um die Frage konkret zu beantworten.
Wenn du die Frage nicht aus dem Kontext beantworten kannst, antworte mit "Das weiß ich leider nicht.", sonst nichts.
Erfinde niemals etwas, was nicht aus dem Kontext hervorgeht.

<|start_header_id|>user<|end_header_id|>
Kontext: {context}
Frage: {query}

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt_template = ChatPromptTemplate.from_template(FH_PROMPT)

In [4]:
LLAMA_LLM = ChatOllama(
    model=LLAMA_MODELL,
    temperature=0,
    num_ctx=10000
    # Kann später reduziert werden, je nachdem wie viel Speicher benötigt wird. Kontext (max. 10 * 504) + Frage + Prompt sollte nie mehr als ca. 8k Tokens haben
)

In [5]:
# Kontexte formatieren vor der Übergabe an das LLM
def keep_relevant_metadata(documents: list[Document]) -> Tuple[List[Document], List[str]]:
    relevant_metadata = [
        "standort",
        "studiengang",
        "abschluss"
    ]
    links = []
    for doc in documents:
        if doc.metadata['link'] not in links:
            links.append(doc.metadata['link'])
        doc.metadata = {key: doc.metadata[key] for key in relevant_metadata}
    return documents, links


def format_page_content(documents: list[Document]) -> str:
    formatted_context = ""
    for doc in documents:
        formatted_context += f"Standort: {doc.metadata['standort']}, Studiengang: {doc.metadata['studiengang']}, Abschluss: {doc.metadata['abschluss']}\n{doc.page_content}\n\n"
    return formatted_context


def prepare_context(retrieved_documents: list[Document]) -> Tuple[str, List[str]]:
    documents, links = keep_relevant_metadata(retrieved_documents)
    prepared_context = format_page_content(documents)
    return prepared_context, links


# Qdrant-Clinet mit mixedbread-ai/deepset-mxbai-embed-de-large-v1, Recursive Chunks
best_client = get_qdrant_client(embeddings=EMBEDDINGS,
                                collection_name=COLLECTION,
                                qdrant_url=QDRANT_URL)

2025-04-16 12:09:23,323 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-04-16 12:09:23,334 [INFO] HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks "HTTP/1.1 200 OK"


In [6]:
LLAMA_ANSWERS_DIR = "./antworten_llama_base_model_negative_rejection"
MAX_DOCUMENTS = 10

In [7]:
for frage in FRAGEN.items():
    frage_id, frage_text = frage
    print(f"Frage {frage_id}: {frage_text}")
    retrieved_documents = retrieve_with_score_without_filters(client=best_client,
                                                           max_documents=MAX_DOCUMENTS,
                                                           query=frage_text)
    documents_only = [doc for doc, _ in retrieved_documents]
    context, links = prepare_context(documents_only)
    prompt = rag_prompt_template.invoke({"context": context, "query": frage_text})
    response = LLAMA_LLM.invoke(prompt)
    answer = response.content
    input_tokens = response.usage_metadata["input_tokens"]
    print(f"Input Tokens: {input_tokens}")
    print(f"Antwort: {answer}\n{'-' * 50}")
    print("Für weitere Informationen besuchen Sie:")
    for link in links[
                :2]:  # Nur die ersten beiden Links anzeigen, da der MRR=0.911 ist, ist davon auszugehen, dass die relevanten Links in den ersten Positionen sind
        print(f"- {link}")

    clean_question_text = re.sub(r'[^\w\s-]', '', frage_text)
    filename = f"{frage_id}_{clean_question_text}.txt"
    filepath = os.path.join(LLAMA_ANSWERS_DIR, filename)

    with open(filepath, "w", encoding="utf-8") as file:
        file.write(f"Frage {frage_id}: {frage_text}\n\n")
        file.write(f"Antwort:\n{answer}\n\n")

2025-04-16 12:09:33,164 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Frage 1: Wer ist der Präsident der USA?


2025-04-16 12:09:44,178 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Input Tokens: 1237
Antwort: Das weiß ich leider nicht.
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/de/studienangebot/index.php
- https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/verlaufsplaene_modulhandbuecher_1/soest_4/fb_eet_3/Modulhandbuch_BA_ETS_ETA_ETP_20_02.2024.pdf
Frage 2: Wie funktioniert der HNSW Algorithmus?


2025-04-16 12:09:45,410 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-04-16 12:10:07,879 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

Input Tokens: 4290
Antwort: Eine Frage, die direkt zum Thema "Algorithmen und Datenstrukturen" passt!

Die Suche in Vektordatenbanken ist ein klassisches Problem der Datenstrukturierung. Hier sind einige Algorithmen, die sich für diese Aufgabe eignen:

1. **Bäume**: Bäume wie Binärsuche- oder AVL-Bäume können effizient zur Suche in Vektordatenbanken verwendet werden.
2. **Hashing**: Hashfunktionen können verwendet werden, um die Daten schnell und effizient zu indexieren und zu suchen.
3. **Suchalgorithmen für B-Treelike-Strukturen**: Algorithmen wie der B+-Tree oder der B*-Tree können zur Suche in großen Datenbanken eingesetzt werden.

Es ist jedoch wichtig zu beachten, dass die Wahl des geeigneten Algorithmus von den spezifischen Anforderungen und Eigenschaften der Vektordatenbank abhängt.
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/media/neu_np/hv_2/dateien_sg_2_4/verlaufsplaene_modulhandbuecher_1/meschede_6/Modul

2025-04-16 12:10:42,840 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-04-16 12:11:02,587 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Input Tokens: 4312
Antwort: Die Öffnungszeiten der Fachhochschule Südwestfalen variieren je nach Standort. 

Für den Standort Iserlohn sind die Öffnungszeiten wie folgt:

- Montags bis Freitags: 07:00 - 19:30 Uhr
- Samstags (nur bei Präsenzterminen im Verbundstudium): 08:00 - 18:00 Uhr

Für den Standort Meschede sind die Öffnungszeiten wie folgt:

- Montags bis Freitags: 07:00 - 21:00 Uhr
- Samstags (vorlesungsfreie Zeit): nur nach Bedarf

Für den Standort Hagen sind die Öffnungszeiten wie folgt:

- Montags bis Freitags: 07:00 - 21:00 Uhr, vorlesungsfreie Zeit: 07:00 - 20:00 Uhr
- Samstags (vorlesungsfreie Zeit): nur nach Bedarf

Es ist ratsam, sich an den jeweiligen Standort zu wenden, um die genauen Öffnungszeiten für morgen zu erfragen.
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/de/studienangebot/index.php
- https://www.fh-swf.de/de/ueber_uns/standorte_4/iserlohn_4/iserlohn_1.php
Frage 5: Welche Studiengänge wer

2025-04-16 12:11:18,859 [INFO] HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-04-16 12:11:38,862 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Input Tokens: 4401
Antwort: Das weiß ich leider nicht.
--------------------------------------------------
Für weitere Informationen besuchen Sie:
- https://www.fh-swf.de/de/studienangebot/index.php
- https://www.fh-swf.de/de/studienangebot/studiengaenge/index.php
